**Check Google Collab resource**

In [1]:
# If using GPU as accelerator
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed May  6 03:08:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# If using TPU as accelerator
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [2]:
# Check RAM usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


**Importing data from Google Drive**

In [3]:
from google.colab import drive, files
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%cd "/content/drive/My Drive/Colab Notebooks/"

/content/drive/My Drive/Colab Notebooks


In [5]:
# Sanity check
!pwd

/content/drive/My Drive/Colab Notebooks


**Importing libraries**

In [0]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display 
from PIL import Image
import h5py

from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import csv

In [0]:
# Custom library
import plotImages
import split_utils

# **Main program**

**Dataset 1:**

10 classes (10 poses)
- 730 training
- 109 testing

In [0]:
TRAIN_DIR = '/content/drive/My Drive/Colab Notebooks/training_set'
TEST_DIR = '/content/drive/My Drive/Colab Notebooks/test_set'

In [0]:
# CNN model 
classifier = Sequential()

classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 10, activation = 'softmax')) # number of classes

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Data Augmentation
batch_size = 32
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range = 20,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [10]:
training_set = train_datagen.flow_from_directory(TRAIN_DIR,
                                                 target_size = (64, 64),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(TEST_DIR,
                                            target_size = (64, 64),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

classifier.fit_generator(training_set,
                         steps_per_epoch = 729 // batch_size, # number of training set images, 729
                         epochs = 1,
                         validation_data = test_set,
                         validation_steps = 229 // batch_size) # number of test set images, 229

Found 730 images belonging to 10 classes.
Found 109 images belonging to 10 classes.
Epoch 1/1
22/22 [==============================] - 220s 10s/step - loss: 2.3485 - accuracy: 0.1791 - val_loss: 2.1465 - val_accuracy: 0.2780


In [0]:
classifier.save('yoga_practice_model_10.h5') #save model 

**Dataset 2:**

107 classes (107 poses)
- 5353 training
- 641 testing

In [0]:
DATA_DIR = '/content/drive/My Drive/Colab Notebooks/dataset_full'

In [12]:


batch_size_train = 32
validation_split = 0.1
colorMode = 'rgb'
classMode = "categorical"
imgSize = 64

# all data in train_dir and val_dir which are alias to original_data. (both dir is temporary directory)
# don't clear base_dir, because this directory holds on temp directory.
base_dir, train_dir, val_dir = split_utils.train_valid_split(DATA_DIR, validation_split, seed=1)

# generator for train data
train_datagen = ImageDataGenerator(
    rescale=1./255,    
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(imgSize, imgSize),
    color_mode=colorMode,
    class_mode=classMode,
    batch_size=batch_size_train,
)

# generator for validation data
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(imgSize, imgSize),
    color_mode=colorMode,
    class_mode=classMode,
    batch_size=batch_size_train,
    )

print('The ratio of validation_split is {}'.format(validation_split))
print('The size of train_dir is {}'.format(train_generator.n))
print('The size of val_dir is {}'.format(validation_generator.n))

Found 5353 images belonging to 107 classes.
Found 641 images belonging to 107 classes.
The ratio of validation_split is 0.1
The size of train_dir is 5353
The size of val_dir is 641


In [17]:
# Model 2: Including dropout layer
classifier2 = Sequential()

classifier2.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier2.add(MaxPooling2D(pool_size = (2, 2)))
classifier2.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier2.add(MaxPooling2D(pool_size = (2, 2)))
classifier2.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier2.add(MaxPooling2D(pool_size = (2, 2)))
classifier2.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier2.add(MaxPooling2D(pool_size = (2, 2)))
classifier2.add(Flatten())
classifier2.add(Dropout(0.5))
classifier2.add(Dense(units = 512, activation = 'relu'))
classifier2.add(Dense(units = 107, activation = 'softmax')) # number of classes

classifier2.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 4, 4, 128)        

In [0]:
filepath_2 = "classifier_2.hdf5"

logpath_2 = "Model_2_history.csv"
log_2 = CSVLogger(logpath_2, append=True)

early_stop = EarlyStopping(monitor='val_accuracy',
                           min_delta=0.001,
                           patience=5)

checkpoint_2 = ModelCheckpoint(filepath = filepath_2, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks_list = [                              
    early_stop,
    checkpoint_2,
    log_2
]

In [0]:
classifier2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])                  


In [0]:
classifier2.fit_generator(
          train_generator,
          steps_per_epoch= 5353 // batch_size_train,                             
          epochs=30,                                                      
          validation_data=validation_generator,
          callbacks = callbacks_list)

Epoch 1/30
116/167 [===================>..........] - ETA: 7:12 - loss: 4.6679 - accuracy: 0.0119

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


167/167 [==============================] - 1467s 9s/step - loss: 4.6654 - accuracy: 0.0118 - val_loss: 4.6085 - val_accuracy: 0.0140

Epoch 00001: val_accuracy improved from -inf to 0.01404, saving model to classifier_2.hdf5
Epoch 2/30
167/167 [==============================] - 45s 267ms/step - loss: 4.6498 - accuracy: 0.0148 - val_loss: 4.6704 - val_accuracy: 0.0140

Epoch 00002: val_accuracy did not improve from 0.01404
Epoch 3/30
167/167 [==============================] - 44s 263ms/step - loss: 4.6359 - accuracy: 0.0133 - val_loss: 4.5749 - val_accuracy: 0.0250

Epoch 00003: val_accuracy improved from 0.01404 to 0.02496, saving model to classifier_2.hdf5
Epoch 4/30
121/167 [====================>.........] - ETA: 11s - loss: 4.5563 - accuracy: 0.0214

In [0]:
classifier2.save('yoga_model_2.h5') #save model 